In [111]:
import pandas as pd 
import numpy as np
airdata = pd.read_csv("datasets/datasets/airquality.csv")

In [3]:
airdata.head()

,Unnamed: 0,Ozone,Solar.R,Wind,Temp,Month,Day
0,1,41.0,190.0,7.4,67,5,1
1,2,36.0,118.0,8.0,72,5,2
2,3,12.0,149.0,12.6,74,5,3
3,4,18.0,313.0,11.5,62,5,4
4,5,NaN,NaN,14.3,56,5,5


In [4]:
len(airdata)

153

In [34]:
airdata.isnull().sum()

Unnamed: 0    0
Ozone         0
Solar.R       7
Wind          0
Temp          0
Month         0
Day           0
dtype: int64

In [6]:
Avgofozone_month = airdata.groupby(['Month'])[["Ozone"]].mean()
Avgofozone = airdata['Ozone'].mean()

In [7]:
Avgofozone_month


,Ozone
Month,
5,23.615385
6,29.444444
7,59.115385
8,59.961538
9,31.448276


In [8]:
airdata.loc[airdata['Month'] == 5].groupby(['Month'])[["Ozone"]].mean()

,Ozone
Month,
5,23.615385


In [28]:
sampl = airdata[['Ozone','Month']]

In [112]:
def month_avg(coll,mont):
    reslt = coll.loc[airdata['Month'] == mont].groupby(['Month'])[["Ozone"]].mean()
    return reslt['Ozone'].round().astype(np.int64)


def age_approx(row):
    Ozone = row['Ozone']
    Month = row['Month']
    
    if pd.isnull(Ozone):
        res = month_avg(airdata[['Ozone','Month']],Month)
        return res.to_frame().reset_index().iloc[0]['Ozone']
    else:
        return Ozone


In [108]:
ne = month_avg(airdata[['Ozone','Month']],5)

In [110]:
ne.to_frame().reset_index().iloc[0]['Ozone']


24

In [113]:
airdata['Ozone'] = airdata.apply(age_approx, axis=1)

In [114]:
airdata['Ozone']
airdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 7 columns):
Unnamed: 0    153 non-null int64
Ozone         153 non-null float64
Solar.R       146 non-null float64
Wind          153 non-null float64
Temp          153 non-null int64
Month         153 non-null int64
Day           153 non-null int64
dtypes: float64(3), int64(4)
memory usage: 8.4 KB


In [115]:
airdata[['Ozone','Month']]

,Ozone,Month
0,41.0,5
1,36.0,5
2,12.0,5
3,18.0,5
4,24.0,5
5,28.0,5
6,23.0,5
7,19.0,5
8,8.0,5
9,24.0,5
